In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from tetris_main import *
import random
from library.hill_climbing.hc import hill_climb
from library.simulated_annealing.sa import simulated_annealing
from library.genetic_algorithm import gao
import pandas as pd

In [28]:
coord = [0, 2]
array = np.array([[0,0], [0,1], [1, 0], [1,1]])
np.sum([np.sum(coord == i) == 2 for i in array])
np.sum(np.all(coord == array, axis=1)) == 0

True

In [2]:
grid = np.zeros((10,10))
grid_shape = (12, 12)
pieces = pieces_generator(grid_shape)
grid, pieces_coordinates = tetrimino_fitter(pieces, grid_shape)

In [3]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 1],
            y=pieces_coordinates[i][:, 0],
            marker_size=30,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=600,
        height=600,
        xaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True, showticklabels=False),
        yaxis=dict(range=[grid.shape[0] - 0.5, -0.5], tickfont_size=12, showticklabels=False),
    ),
)
fig.write_html(f"graphs/test_{grid.size}_size.html")
# fig.write_image(f"graphs/test_{grid.size}_size.png", scale=2)
fig.show()


In [4]:
pop_size = 3
pieces_list = pieces_generator(grid_shape, rotation=False)
# Generate each individual with hill climbing
individuals = [generate_individual(pieces_list, grid_shape) for i in range(pop_size)]

In [6]:
pop = Population(individuals.copy(), "max", 2, pieces_list, grid_shape)
gao.GAO(
    pop,
    50,
    tournament_size=3,
    p_crossover=0.01,
    p_mutation=0.2,
    n_mutations=1,
    p_rot_mut=0.2,
    p_adoption=0.01,
)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)

In [16]:
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 1],
            y=pieces_coordinates[i][:, 0],
            marker_size=30,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        margin=dict(l=00, r=00, t=80, b=0),
        width=600,
        height=600,
        title=f"<b>Fitness:</b> {int(pop.elites[0].fitness)}/{int(np.sum(100*np.ones_like(grid)))}<br><b>Pieces used:</b> {len(pieces_coordinates)}/{len(pieces_list)}",
        xaxis=dict(range=[-0.5, grid.shape[1] - 0.5], fixedrange=True,showticklabels=False),
        yaxis=dict(range=[-0.5, grid.shape[0] - 0.5], tickfont_size=12,showticklabels=False),
    ),
)
fig.update_layout(showlegend=False)
fig.write_html(f"graphs/test_ga_{grid.size}_size.html")

fig.show()

In [29]:
fitness_df = pd.DataFrame(pop.fitness_history, columns=["Fitness"])
fitness_df[:5]

,Fitness
0,13200.0
1,13200.0
2,13200.0
3,13200.0
4,13200.0


In [67]:
fig = px.line(fitness_df, title="", labels={"index": "Generation", "value": "Fitness"}, width=600, height=300, template="seaborn")
fig.update_layout(font_family="Helvetica", yaxis_tickformat="0", margin=dict(l=60, r=40, t=20, b=0))
fig

In [ ]:
# Threshold Acceptance Algorithm

In [7]:
# Hill Climbing
pop = Population(individuals, "max", 1, pieces_list, grid_shape)
hill_climb(pop, hardstop=1)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()

In [8]:
# Simulated Annealing
pop = Population(individuals, "max", 1, pieces_list, grid_shape)
simulated_annealing(pop)
grid, pieces_coordinates = tetrimino_fitter(pop.elites[0].representation, grid_shape)
fig = go.Figure(
    data=[
        go.Scatter(
            x=pieces_coordinates[i][:, 0],
            y=pieces_coordinates[i][:, 1],
            marker_size=50,
            marker_symbol="square",
            mode="markers",
        )
        for i in range(len(pieces_coordinates))
    ],
    layout=dict(
        width=800,
        height=800,
        title=f"Fitness: {pop.elites[0].fitness}",
        yaxis=dict(
            range=[-0.5, grid.shape[1]-.5],
            fixedrange=True,
        ),
        xaxis=dict(
            range=[-0.5, grid.shape[0]-.5],
            tickfont_size=12,
        ),
    ),
)
fig.show()